In [1]:
!pip install bs4 tqdm

In [5]:
import requests
from bs4 import BeautifulSoup
import re
import os
from concurrent.futures import ThreadPoolExecutor
import time
from tqdm import tqdm  # For progress bar
import math
import zipfile
from osgeo import ogr
import json


## Harvest Zip URLs

In [ ]:


# URL of the page to scrape
url = "https://disasters.geoplatform.gov/USA_Structures/"

# Send a GET request to fetch the HTML content of the page
response = requests.get(url)

# Save the HTML content to a local file
with open('USA_Structures.html', 'w', encoding='utf-8') as file:
    file.write(response.text)

# Parse the HTML content using BeautifulSoup
soup = BeautifulSoup(response.text, 'html.parser')

# Find all URLs that end with .zip (assuming these are the downloadable zip files)
zip_links = []
for link in soup.find_all('a', href=True):
    if re.search(r'\.zip$', link['href']):
        zip_links.append(link['href'])

# Print the list of zip file URLs
for zip_link in zip_links:
    print(zip_link)
    
print(len(zip_links))



# Download to Q Drive

In [ ]:
# Destination folder where the files will be saved (SMB path)
destination_folder = "/Volumes/GIS/FEMA_USA_Structures_10_17_2024"

# Ensure the destination folder exists
os.makedirs(destination_folder, exist_ok=True)

# Function to download a single file with progress feedback
def download_zip(url, index, total_files):
    start_time = time.time()
    file_name = os.path.join(destination_folder, os.path.basename(url))
    response = requests.get(url, stream=True)
    
    if response.status_code == 200:
        total_size = int(response.headers.get('content-length', 0))
        block_size = 1024  # 1 Kilobyte
        t = tqdm(total=total_size, unit='B', unit_scale=True, desc=f"File {index+1}/{total_files}: {file_name}")

        with open(file_name, 'wb') as f:
            for data in response.iter_content(block_size):
                t.update(len(data))
                f.write(data)
        t.close()

        # Time calculation for this file
        download_time = time.time() - start_time
        file_size_MB = total_size / (1024 * 1024)
        print(f"Downloaded: {file_name} | Size: {file_size_MB:.2f} MB | Time: {download_time:.2f} s")

    else:
        print(f"Failed to download: {url}")

# Function to download files in parallel with a control on simultaneous downloads
def download_zips_in_parallel(zip_links, max_workers=4):
    total_files = len(zip_links)
    start_time = time.time()
    
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = []
        for index, url in enumerate(zip_links):
            futures.append(executor.submit(download_zip, url, index, total_files))

    # Total time calculation
    total_time_elapsed = time.time() - start_time
    print(f"All downloads completed in {total_time_elapsed:.2f} seconds")

# Call the function to start downloading with progress
download_zips_in_parallel(zip_links, max_workers=4)


# Unzip folders

In [ ]:


# Path to the folder containing the zip files
folder_path = "/Volumes/GIS/FEMA_USA_Structures_10_17_2024"

# Function to unzip all zip files in the folder
def unzip_files_in_folder(folder_path):
    # Iterate through all files in the folder
    for file_name in os.listdir(folder_path):
        # Check if the file is a zip file
        if file_name.endswith('.zip'):
            file_path = os.path.join(folder_path, file_name)
            # Define the directory where the files will be extracted
            extract_path = folder_path
            
            # Unzip the file
            with zipfile.ZipFile(file_path, 'r') as zip_ref:
                print(f"Unzipping {file_name}...")
                zip_ref.extractall(extract_path)
                print(f"Unzipped {file_name} to {extract_path}")

# Call the function to start unzipping
unzip_files_in_folder(folder_path)


# Convert GDB to GEoJSON with Fiona

In [ ]:
# Path to the root folder where GDB files are stored
root_folder = "/Volumes/GIS/FEMA_USA_Structures_10_17_2024"

# Function to convert feature layer to newline-delimited GeoJSON
def convert_to_ndjson(input_gdb, layer_name, output_file):
    # Open the GDB layer using Fiona
    with fiona.open(input_gdb, layer=layer_name) as source:
        with open(output_file, 'w') as output_ndjson:
            # Iterate through each feature and write it in GeoJSON format
            for feature in source:
                output_ndjson.write(json.dumps(feature) + '\n')
            print(f"Converted {layer_name} to {output_file}")

# Function to traverse directories and find GDB files
def traverse_and_convert(root_folder):
    # Walk through the root folder
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for dirname in dirnames:
            if dirname.endswith('.gdb'):
                gdb_path = os.path.join(dirpath, dirname)
                print(f"Found GDB: {gdb_path}")

                # List all layers in the GDB
                with fiona.Env():
                    layers = fiona.listlayers(gdb_path)
                    for layer in layers:
                        print(f"Processing layer: {layer}")
                        # Define output path for the newline-delimited GeoJSON
                        output_ndjson = os.path.join(dirpath, f"{layer}.ndjson")
                        # Convert the feature layer to newline-delimited GeoJSON
                        convert_to_ndjson(gdb_path, layer, output_ndjson)

# Start the traversal and conversion process
traverse_and_convert(root_folder)


# Use GDAL to Convert GDB to Geojson

In [ ]:

# Path to the root folder where GDB files are stored
root_folder = "/Volumes/GIS/FEMA_USA_Structures_10_17_2024"

# Function to convert feature layer to newline-delimited GeoJSON
def convert_to_ndjson(input_gdb, layer_name, output_file):
    driver = ogr.GetDriverByName('OpenFileGDB')
    gdb = driver.Open(input_gdb, 0)  # 0 means read-only
    if not gdb:
        print(f"Failed to open GDB: {input_gdb}")
        return

    layer = gdb.GetLayerByName(layer_name)
    if not layer:
        print(f"Layer {layer_name} not found in GDB: {input_gdb}")
        return

    with open(output_file, 'w') as output_ndjson:
        # Iterate through features in the layer
        for feature in layer:
            geojson_feature = feature.ExportToJson()
            output_ndjson.write(geojson_feature + '\n')
        print(f"Converted {layer_name} to {output_file}")

# Function to traverse directories and find GDB files
def traverse_and_convert(root_folder):
    # Walk through the root folder
    for dirpath, dirnames, filenames in os.walk(root_folder):
        for dirname in dirnames:
            if dirname.endswith('.gdb'):
                gdb_path = os.path.join(dirpath, dirname)
                print(f"Found GDB: {gdb_path}")

                # Open GDB to find available layers
                driver = ogr.GetDriverByName('OpenFileGDB')
                gdb = driver.Open(gdb_path, 0)
                if gdb:
                    for i in range(gdb.GetLayerCount()):
                        layer = gdb.GetLayerByIndex(i)
                        layer_name = layer.GetName()
                        print(f"Processing layer: {layer_name}")

                        # Define output path for the newline-delimited GeoJSON
                        output_ndjson = os.path.join(dirpath, f"{layer_name}.ndjson")
                        # Convert the feature layer to newline-delimited GeoJSON
                        convert_to_ndjson(gdb_path, layer_name, output_ndjson)

# Start the traversal and conversion process
traverse_and_convert(root_folder)
